In [11]:
#Libraries 
#pip install 'gymnasium[box2d]' pygame matplotlib box2d
import gymnasium as gym
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import numpy as np
import utils
import time

In [2]:
#Creating env 
env = gym.make("LunarLander-v3", render_mode="rgb_array")

In [3]:
#Adding seed so it always stays same
env.action_space.seed(42)

42

In [4]:
# Observation space
print("Observation space:", env.observation_space)
print("Shape of state vector:", env.observation_space.shape)

Observation space: Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)
Shape of state vector: (8,)


In [5]:
# Print example states
print("\nExample state samples:")
for _ in range(3):
    print(env.observation_space.sample())


Example state samples:
[-1.9009957  -2.2756608   8.252366   -6.984867    4.2583613   7.250959
  0.13593407  0.40009543]
[ 0.9890819   0.51520437 -8.563686   -3.2265618  -3.2137537   2.566121
  0.33255854  0.83416903]
[ 1.0608166  -0.6380764   2.0276923   2.3738945   1.601922    4.4841666
  0.22414678  0.38137743]


In [6]:

# Action space
print("\nAction space:", env.action_space)
print("Number of discrete actions:", env.action_space.n)


Action space: Discrete(4)
Number of discrete actions: 4


In [7]:
# Print all possible actions
print("\nAll possible actions:")
for a in range(env.action_space.n):
    print(f"Action {a}")



All possible actions:
Action 0
Action 1
Action 2
Action 3


In [8]:

# Reset the environment and get the initial state.
current_state = env.reset()

# Select an action
action = 0

# Run a single time step of the environment's dynamics with the given action.
next_state, reward, terminated, truncated, info = env.step(action)
done = terminated or truncated

print(f"{'Current State:':<15} {current_state}")
print(f"{'Action Taken:':<15} {action}")
print(f"{'Next State:':<15} {next_state}")
print(f"{'Reward:':<15} {reward}")
print(f"{'Done:':<15} {done}")


# Replace the `current_state` with the state after the action is taken
current_state = next_state

Current State:  (array([-0.00278196,  1.4147277 , -0.28180486,  0.16921836,  0.00323045,
        0.06383304,  0.        ,  0.        ], dtype=float32), {})
Action Taken:   0
Next State:     [-0.00556431  1.4179573  -0.28142354  0.14352864  0.00638476  0.0630924
  0.          0.        ]
Reward:         0.640480647797915
Done:           False


## Implementing Q learning


In [ ]:
# records the current time to later compute how long training took.
start = time.time()

In [12]:

#Total number of episodes (training iterations).
#Each episode represents a full run from initial state to terminal state (or until the timestep limit).
#The agent learns from each episode.
num_episodes = 100

#Maximum steps allowed per episode.
#Prevents infinite loops in non-terminal environments.
max_num_timesteps = 50

#Stores the cumulative reward (or "points") obtained in each episode.
total_point_history = []

#Used to compute a moving average of the total points over the last 10 episodes.
num_p_av = 10    # number of total points to use for averaging

#With probability ε, choose a random action (exploration).
#With probability 1-ε, choose the best known action (exploitation).
epsilon = 1.0     # initial ε value for ε-greedy policy


In [ ]:
#1 Initialize memory buffer D with capacity N
memory_length = 10000
#N = capacity of the buffer, i.e., how many experiences it can store.
#deque comes from Python’s collections module: from collections import deque.
#It's a data structure like a list but optimized for fast appends and pops from both ends.
#Think of it as a queue with automatic memory management when a max length is set.
memory_buffer = deque(maxlen=memory_length)


#2 Initialize Q-Network with random weights w
q_network = Sequential([ 
    tf.keras.layers.Dense(24, activation='relu',input_shape=(state_size)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(units=num_actions, activation='linear') 
    ])


#3 Initialize target Q-Network with weights w = w
q_target = Sequential([ 
    tf.keras.layers.Dense(24, activation='relu',input_shape=(state_size)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(units=num_actions, activation='linear') 
    ])

#Making both network weight same 
q_target.set_weights(q_network.get_weights())

In [ ]:
#4 for episode i = 1 to M do
for i in range(num_episodes):
    #5 Receive initial observation state S ₁
    # Reset the environment and get the initial state.
    state = env.reset()
    total_points = 0 
    
     #for t=1 to T do Observe state St and choose action At using an e-greedy policy
    for t in range(max_num_timesteps):
        #creating a state_qn from state dimensions
        state_qn = np.expand_dims(state, axis=0)
        
        '''
        Passes the (expanded) state through the Q-network.

        The Q-network predicts Q-values for all possible actions from this state.

        Output is typically a 1D array: e.g. [Q(s,a₀), Q(s,a₁), ..., Q(s,aₙ)]
        '''
        q_values = q_network(state_qn)

        #Choose an action using ε-greedy strategy.
        action = utils.get_action(q_values, epsilon)
        
        #Take action At in the environment , receive reward Rt and next state St+1
        next_state, reward, terminated, truncated, info = env.step(action)
        
        #Store experience tuple ( St , At , Rt , St+1 ) in memory buffer D
        
        
10 Every C steps perform a learning update :
11 Sample random mini-batch of experience tuples ( Sj , Aj , Rj , Sj+1 ) from D
12 Set yj = R ; if episode terminates at step j + 1 , otherwise set yj = R + maxa ' Q(sj+1 , α ' )
13 Perform a gradient descent step on ( yjQ(sj , aj ; w))2 with respect to the Q-Network weights w
14 Update the weights of the Q-Network using a soft update
15 end
16 end

SyntaxError: invalid character '₁' (U+2081) (3647325864.py, line 20)